# Trabajo Práctico 2: Predicción de tormenta en base de Datos Meteorológicos 

### Integrantes

- Andrioli, Facundo
- Pérez, José

### Preparación de Datos: 

1. Lea el dataset limpio que se guardó en el TP1 (punto 2 de conclusiones)

In [1]:
import pandas as pd

# Cargamos el conjunto de datos
file = 'weatherAUS_output.csv'
df = pd.read_csv(file)

2. Realice una codificación adecuada de variables categóricas si es necesario, utilizando técnicas como One-Hot 
Encoding según corresponda. 

In [2]:
# Identificamos las columnas categóricas
categorical_columns = df.select_dtypes(include=['object']).columns

# Aplicamos One-Hot Encoding a las columnas categóricas
df_encoded = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Mostramos las primeras filas del dataset transformado
print(df_encoded.head())

   MinTemp  MaxTemp  Rainfall  Evaporation  Sunshine  WindGustSpeed  \
0     12.7     15.8       0.8          1.4       7.8           35.0   
1      6.2     15.1       0.0          1.8       2.1           20.0   
2      5.3     15.9       0.0          1.4       8.0           30.0   
3     11.3     15.7       0.0          1.4       1.5           52.0   
4      7.6     11.2      16.2          4.6       1.1           46.0   

   WindSpeed9am  WindSpeed3pm  Humidity9am  Humidity3pm  ...  WindDir3pm_NNW  \
0          13.0          15.0         75.0         52.0  ...           False   
1           2.0          11.0         81.0         56.0  ...           False   
2           6.0          13.0         71.0         46.0  ...           False   
3          15.0          22.0         62.0         62.0  ...            True   
4          17.0          13.0         83.0         88.0  ...           False   

   WindDir3pm_NW  WindDir3pm_S  WindDir3pm_SE  WindDir3pm_SSE  WindDir3pm_SSW  \
0          

3. Separe el dataset en entrenamiento y prueba (70% entrenamiento, 30% prueba).

In [3]:
from sklearn.model_selection import train_test_split

# Definimos las características (X) y el objetivo (y)
X = df_encoded.drop('RainTomorrow', axis=1)
y = df_encoded['RainTomorrow']

# Separamos el dataset en entrenamiento (70%) y prueba (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Datos de train y de test
print("Valores de X_train:",X_train.size)
print("Valores de y_train:",y_train.size)
print("Valores de X_test:",len(X_test))
print("Valores de y_test:",len(y_test))

Valores de X_train: 352834092
Valores de y_train: 99502
Valores de X_test: 42644
Valores de y_test: 42644


4. Normalice o estandarice los atributos numéricos para asegurar que todas las variables tengan la misma escala.

In [4]:
from sklearn.preprocessing import StandardScaler

# Identificamos las columnas numéricas
numerical_columns = X_train.select_dtypes(include=['float64', 'int64']).columns

# Creamos una instancia del StandardScaler
scaler = StandardScaler()

# Ajustamos el escalador a los datos de entrenamiento y transformamos los datos de entrenamiento
X_train[numerical_columns] = scaler.fit_transform(X_train[numerical_columns])

# Utilizamos el mismo escalador para transformar los datos de prueba
X_test[numerical_columns] = scaler.transform(X_test[numerical_columns])

# Mostramos las primeras filas del conjunto de entrenamiento transformado
print(X_train.head())


        MinTemp   MaxTemp  Rainfall  Evaporation  Sunshine  WindGustSpeed  \
5426  -0.728788 -1.024421 -0.136110    -0.529909 -0.727728      -1.583889   
75590  1.160302  0.576340 -0.277473    -0.529909 -1.276662       0.281685   
9368   0.098664  1.250345 -0.277473     0.171888  1.111201       0.558067   
73772 -0.822462 -0.813794 -0.277473    -0.529909 -1.276662       0.212590   
3398   0.286012 -0.855920 -0.230352    -0.572442 -1.249215      -1.583889   

       WindSpeed9am  WindSpeed3pm  Humidity9am  Humidity3pm  ...  \
5426      -1.572602      2.009055     1.458635     0.935639  ...   
75590      0.113636      0.794151     0.783584     1.270228  ...   
9368      -0.336027      0.794151    -2.955154    -2.123465  ...   
73772      0.563299      0.131476    -0.203027    -0.880704  ...   
3398       1.575042      1.015042     0.523950     1.557019  ...   

       WindDir3pm_NNW  WindDir3pm_NW  WindDir3pm_S  WindDir3pm_SE  \
5426            False          False         False         

### Modelado y Evaluación:

1. Determine dos métricas de evaluación que considere importante medir para este problema. Justifique su elección.

#### Error cuadrático medio (MSE)

El MSE mide la media de los cuadrados de los errores, es decir, la diferencia entre los valores predichos y los valores reales. Penaliza los errores grandes más severamente que los pequeños debido al cuadrado de los errores. Esto es útil cuando quieres evaluar la precisión de las predicciones y el impacto de errores grandes en el modelo.

#### Error absoluto medio porcentual (MAPE)

El MAPE mide el error absoluto en términos porcentuales respecto al valor real. Es decir, calcula el promedio de las diferencias absolutas entre los valores predichos y los valores reales, expresadas como un porcentaje del valor real. Esta métrica es útil porque ofrece una forma de evaluar la precisión del modelo en términos relativos, haciendo más fácil comparar el rendimiento del modelo entre diferentes escalas de datos o diferentes conjuntos de datos.

2. Implemente un modelo de base usando como predicción que determine que llueve mañana si hoy llueve, y si hoy no llueve
mañana no va a llover. 

In [5]:
from sklearn.metrics import accuracy_score, confusion_matrix

def base_model_prediction(X):
  return X['RainToday'].astype(int)

# Predecir con el modelo de base
y_pred_base = base_model_prediction(X_test) 

# Evaluar el modelo de base
accuracy = accuracy_score(y_test, y_pred_base)
conf_matrix = confusion_matrix(y_test, y_pred_base)

print(f"Exactitud del modelo base: {accuracy}")
print("Matriz de confusión:")
print(conf_matrix)

Exactitud del modelo base: 0.7605759309633243
Matriz de confusión:
[[27997  5141]
 [ 5069  4437]]


3. Implemente y entrene un modelo de k Nearest Neighbors (kNN) para predecir si va a llover mañana. Utilice la 
distancia euclidiana como medida de distancia entre vecinos.Ajuste el parámetro k utilizando técnicas como la 
validación cruzada (5 folds) para optimizar el rendimiento del modelo (utilice como función de optimización una de las 
métricas definidas en el punto anterior). 

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.metrics import make_scorer, mean_squared_error, classification_report

# Creamos un imputador que reemplace los NaN por la media de cada columna
imputer = SimpleImputer(strategy='mean')

# Aplicamos el imputador a los datos de entrenamiento y prueba
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

print("Definiendo el clasificador kNN...")
knn = KNeighborsClassifier(metric='euclidean')

# Definimos la métrica MSE para GridSearchCV
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

# Definimos los parámetros para GridSearchCV
param_grid = {'n_neighbors': [1, 3, 5, 7, 9, 11]}  # Probar valores de k impares
print("Iniciando GridSearchCV para encontrar el mejor valor de k...")

# Usamos GridSearchCV para encontrar el mejor k usando validación cruzada (5 folds)
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring=mse_scorer, verbose=3)
grid_search.fit(X_train_imputed, y_train)

# Mostramos el mejor parámetro k y el mejor puntaje
best_k = grid_search.best_params_['n_neighbors']
best_score = grid_search.best_score_
print(f"\nMejor valor de k encontrado: {best_k}")
print(f"Puntaje de validación cruzada (accuracy) con el mejor k: {best_score:.2f}")

# Entrenamos el modelo kNN con el mejor valor de k en el conjunto de entrenamiento
print("Entrenando el modelo kNN con el mejor valor de k...")
best_knn = KNeighborsClassifier(n_neighbors=best_k, metric='euclidean')
best_knn.fit(X_train_imputed, y_train)
print("Modelo entrenado.")

# Hacemos predicciones en el conjunto de prueba
print("Haciendo predicciones en el conjunto de prueba...")
y_pred = best_knn.predict(X_test_imputed)

# Evaluar el modelo
class_report = classification_report(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Reporte de clasificación
print("Reporte de Clasificación:")
print(class_report)

# Mostramos los resultados
print(f"Exactitud del modelo: {accuracy}")
print("Matriz de confusión del modelo:")
print(conf_matrix)



Definiendo el clasificador kNN...
Iniciando GridSearchCV para encontrar el mejor valor de k...
Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END ....................n_neighbors=1;, score=-0.196 total time= 4.7min
[CV 2/5] END ....................n_neighbors=1;, score=-0.194 total time= 4.4min
[CV 3/5] END ....................n_neighbors=1;, score=-0.198 total time= 4.4min
[CV 4/5] END ....................n_neighbors=1;, score=-0.199 total time= 4.5min
[CV 5/5] END ....................n_neighbors=1;, score=-0.198 total time= 4.4min
[CV 1/5] END ....................n_neighbors=3;, score=-0.174 total time= 4.4min
[CV 2/5] END ....................n_neighbors=3;, score=-0.168 total time= 4.4min
[CV 3/5] END ....................n_neighbors=3;, score=-0.171 total time= 4.5min
[CV 4/5] END ....................n_neighbors=3;, score=-0.172 total time= 4.3min
[CV 5/5] END ....................n_neighbors=3;, score=-0.174 total time= 4.3min
[CV 1/5] END ....................n_

4. Implemente y entrene un modelo de regresión logística para predecir la misma variable objetivo.

In [7]:
from sklearn.linear_model import LogisticRegression

# Creamos una instancia del modelo de regresión logística
log_reg = LogisticRegression(max_iter=1000, random_state=42)

# Entrenamos el modelo con los datos de entrenamiento imputados
log_reg.fit(X_train_imputed, y_train)

# Predecir en el conjunto de prueba
y_pred_log_reg = log_reg.predict(X_test_imputed)

# Evaluar el modelo
accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)
conf_matrix_log_reg = confusion_matrix(y_test, y_pred_log_reg)

# Mostramos los resultados
print(f"Exactitud del modelo de regresión logística: {accuracy_log_reg}")
print("Matriz de confusión del modelo de regresión logística:")
print(conf_matrix_log_reg)

Exactitud del modelo de regresión logística: 0.8505768689616359
Matriz de confusión del modelo de regresión logística:
[[31174  1964]
 [ 4408  5098]]


5. Evalúe los dos modelos con las métricas definidas al principio. Discuta las fortalezas y debilidades de cada enfoque 
en el contexto específico de este problema comparándolos con el modelo de base.

### Conclusiones:

1. Resuma los hallazgos más relevantes obtenidos.

2. Guarde los conjuntos de entrenamiento y evaluación para el siguiente TP en archivos `csv`.

3. Guarde en formato pickle el mejor modelo, seleccionado según su criterio.